In [8]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
print("Model loaded successfully!")


Model loaded successfully!


In [12]:
import numpy as np
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter

from sentence_transformers import SentenceTransformer
from pinecone import Pinecone

# Initialize Pinecone using the new approach
pc = Pinecone(api_key="pcsk_3rWW1w_Eua9C9tD1rbQybpChVD9nDijUycon7auXNs3afy7T2Z2zK2YnSHEFeLmKJsx4pp", region="us-east-1")

# Create or connect to your index
index_name = "video-embeddings"
index = pc.Index(index_name)

# Load Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

def extract_transcript(video_id):
    """Extracts transcript from a YouTube video, using both manually uploaded and automatic captions."""
    try:
        # Try fetching the regular subtitles first
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        formatter = TextFormatter()
        return formatter.format_transcript(transcript)
    
    except Exception as e:
        print(f"Error fetching transcript for video {video_id}: {e}")
    
    # Fallback to automatic captions if available
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'], is_automatic=True)
        formatter = TextFormatter()
        return formatter.format_transcript(transcript)
    except Exception as e:
        print(f"Error fetching automatic captions for video {video_id}: {e}")
        return None

def get_embedding(text):
    """Generates embedding using Sentence Transformers."""
    return model.encode(text).tolist()

def generate_embeddings(video_id):
    """Generates embeddings from a YouTube video transcript."""
    transcript = extract_transcript(video_id)
    if not transcript:
        return None

    lines = transcript.split("\n")
    chunk_size = 5  # Adjust chunk size
    text_chunks = [" ".join(lines[i:i+chunk_size]) for i in range(0, len(lines), chunk_size)]

    embeddings = np.array([get_embedding(chunk) for chunk in text_chunks], dtype=np.float32)
    return embeddings

def store_embeddings_in_pinecone(embeddings, video_id):
    """Store embeddings in Pinecone index."""
    if embeddings is not None:
        # Create unique IDs for each embedding (e.g., using video ID and chunk index)
        ids = [f"{video_id}_{i}" for i in range(len(embeddings))]
        
        # Prepare data in the format Pinecone expects
        data = list(zip(ids, embeddings.tolist()))

        # Upsert the embeddings into the Pinecone index
        index.upsert(vectors=data)
        print(f"Successfully upserted {len(data)} embeddings into Pinecone.")

if __name__ == "__main__":
    yt_video_id = "PSs6nxngL6k"  # Replace with actual YouTube video ID
    embeddings = generate_embeddings(yt_video_id)

    # Store embeddings in Pinecone if available
    store_embeddings_in_pinecone(embeddings, yt_video_id)


Successfully upserted 73 embeddings into Pinecone.
